<a href="https://colab.research.google.com/github/kimdonggyu2008/24-AI-Advance/blob/main/%5BWeek_11%5D_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-augmented generation (RAG)

## Overview

### What is RAG?

RAG는 비공개 또는 실시간 데이터를 추가하여 학습자의 지식을 보강하는 기법입니다.

인공신경망은 광범위한 주제에 대해 추론할 수 있지만, 그 지식은 학습된 특정 시점까지의 공개 데이터로 제한됩니다. 비공개 데이터나 모델의 마감일 이후에 도입된 데이터에 대해 추론할 수 있는 AI 애플리케이션을 구축하려면 모델에 필요한 특정 정보로 모델에 대한 지식을 보강해야 합니다. 적절한 정보를 가져와서 모델 프롬프트에 삽입하는 프로세스를 검색 증강 생성(RAG)이라고 합니다.

### What's in this guide?

LangChain에는 RAG 애플리케이션을 구축하는 데 도움이 되도록 특별히 설계된 여러 구성 요소가 있습니다. 이러한 구성 요소에 익숙해지기 위해 텍스트 데이터 소스에 대한 간단한 질문-답변 애플리케이션을 구축해 보겠습니다. 특히, 릴리안 웡의 블로그 게시물 [LLM 기반 자율 에이전트](https://lilianweng.github.io/posts/2023-06-23-agent/)를 통해 QA 봇을 구축해 보겠습니다. 그 과정에서 일반적인 QA 아키텍처를 살펴보고, 관련 LangChain 구성 요소에 대해 논의하며, 고급 QA 기술을 위한 추가 리소스를 강조할 것입니다. 또한, 애플리케이션을 추적하고 이해하는 데 LangSmith가 어떻게 도움이 되는지 살펴볼 것입니다. 애플리케이션의 복잡성이 증가함에 따라 LangSmith는 점점 더 유용해질 것입니다.

**Note**
여기서는 비정형 데이터를 위한 RAG에 초점을 맞춥니다.

## Architecture

일반적인 RAG 애플리케이션에는 두 가지 주요 구성 요소가 있습니다:

**Indexing**: 소스에서 데이터를 수집하고 인덱싱하기 위한 파이프라인. *이 작업은 보통 오프라인에서 이루어집니다.

**Retrieval and generation**: 런타임에 사용자 쿼리를 받아 인덱스에서 관련 데이터를 검색한 다음 이를 모델로 전달하는 실제 RAG 체인.

Raw data에서 답변에 이르는 가장 일반적인 전체 시퀀스는 다음과 같습니다:

#### Indexing
1. **Load**: 먼저 데이터를 로드해야 합니다. 이를 위해 문서 로더를 사용하겠습니다.
2. **Split**: 텍스트 분할기는 큰 '문서'를 작은 덩어리로 나눕니다. 큰 덩어리는 검색하기 어렵고 모델의 한정된 컨텍스트 창에서는 검색되지 않으므로 이 기능은 데이터를 색인하고 모델에 전달할 때 유용합니다.
3. **Store**: 나중에 검색할 수 있도록 분할을 저장하고 색인할 곳이 필요합니다. 이 작업은 대개 벡터스토어와 임베딩 모델을 사용해 수행합니다.

#### Retrieval and generation
4. **Retrieve**: 사용자 입력이 주어지면 Retriever를 사용하여 스토리지에서 관련 스플릿을 검색합니다.
5. **Generate**: ChatModel / LLM은 질문과 검색된 데이터가 포함된 프롬프트를 사용하여 답변을 생성합니다.

## Setup

### Dependencies

이 단계별 안내에서는 OpenAI 채팅 모델과 임베딩, Chroma 벡터 스토어를 사용하지만, 여기에 표시된 모든 내용은 모든 ChatModel 또는 LLM에서 작동합니다, 임베딩, 벡터스토어 또는 retrievers를 사용합니다.

필요 패키지 설치:

In [ ]:
!pip install -U langchain openai chromadb langchainhub bs4 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0

`OPENAI_API_KEY` 입력하기

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] =

## Quickstart

릴리안 웡의 LLM 기반 자율 에이전트 블로그 게시물을 통해 QA 앱을 구축하고자 한다고 가정해 보겠습니다.

이를 위한 간단한 파이프라인을 약 20줄의 코드로 만들 수 있습니다:

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.2 MB/s eta 0:00:00


In [ ]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
# cleanup
vectorstore.delete_collection()

## Detailed walkthrough

위의 코드를 단계별로 살펴보고 무슨 일이 일어나고 있는지 실제로 이해해 보겠습니다.

## Step 1. Load

먼저 블로그 게시물 콘텐츠를 로드해야 합니다. 이를 위해 소스에서 데이터를 `Documents`로 로드하는 객체인 `DocumentLoader`를 사용할 수 있습니다.  `Documents`는 `page_content`(문자열) 및 `metadata`(딕셔너리) 속성을 가진 객체입니다.

이 경우 `urllib`와 `BeautifulSoup`을 사용하여 전달된 웹 URL을 로드하고 구문 분석하여 URL당 하나의 `Document`를 반환하는 `WebBaseLoader`를 사용하겠습니다. 우리는 `bs_kwargs`를 통해 `BeautifulSoup` 구문 분석기에 매개변수를 전달하여 html -> 텍스트 구문 분석을 사용자 정의할 수 있습니다([BeautifulSoup 문서](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup) 참조). 이 경우 클래스가 "post-content", "post-title" 또는 "post-header"인 HTML 태그만 관련이 있으므로 다른 태그는 모두 제거합니다.

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    },
)
docs = loader.load()

In [ ]:
len(docs[0].page_content)

In [ ]:
print(docs[0].page_content[:500])

## Step 2. Split

로드된 문서의 길이가 42,000자가 넘습니다. 이는 많은 모델의 컨텍스트 창에 맞추기에는 너무 깁니다. 또한 컨텍스트 창에 전체 게시물을 넣을 수 있는 모델의 경우에도 경험적으로 모델은 매우 긴 프롬프트에서 관련 컨텍스트를 찾는 데 어려움을 겪습니다.

그래서 우리는 'Document'를 임베딩과 벡터 저장을 위해 청크로 분할할 것입니다. 이렇게 하면 런타임에 블로그 게시물에서 가장 관련성이 높은 부분만 검색하는 데 도움이 됩니다.

이 경우 문서를 1000자의 청크로 분할하고 청크 간에 200자의 겹침이 있습니다. 겹침은 문장과 관련된 중요한 문맥에서 문장이 분리될 가능성을 줄이는 데 도움이 됩니다. 각 청크가 적절한 크기가 될 때까지 공통 구분 기호(예: 줄 바꿈)를 사용하여 문서를 (재귀적으로) 분할하는 `RecursiveCharacterTextSplitter`를 사용합니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
len(all_splits[0].page_content)

In [ ]:
all_splits[10].metadata

## Step 3. Store

이제 66개의 텍스트 청크가 메모리에 저장되었으므로 나중에 RAG 앱에서 검색할 수 있도록 이를 저장하고 색인을 생성해야 합니다. 이를 수행하는 가장 일반적인 방법은 각 문서 분할의 내용을 임베드하고 해당 임베딩을 벡터 스토어에 업로드하는 것입니다.

그런 다음, 분할을 검색하고 싶을 때 검색 쿼리를 가져와서 임베딩하고 일종의 '유사성' 검색을 수행하여 쿼리 임베딩과 가장 유사한 임베딩을 가진 저장된 분할을 식별합니다. 가장 간단한 유사성 측정은 코사인 유사성으로, 각 임베딩 쌍 사이의 각도의 코사인(매우 높은 차원의 벡터)을 측정합니다.

`Chroma` 벡터 저장소(vector store)와 `OpenAIEmbeddings` 모델을 사용하여 단일 명령으로 모든 문서 분할을 임베드하고 저장할 수 있습니다.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

## Step 4. Retrieve

이제 실제 애플리케이션 로직을 작성해 보겠습니다. 사용자가 질문을 하고, 그 질문과 관련된 문서를 검색하고, 검색된 문서와 초기 질문을 모델에 전달하고, 마지막으로 답변을 반환하는 간단한 애플리케이션을 만들고자 합니다.

LangChain은 문자열 쿼리가 주어지면 관련 문서를 반환할 수 있는 인덱스를 래핑하는 `Retriever` 인터페이스를 정의합니다. 모든 리트리버는 공통 메서드인 `get_relevant_documents()`(및 그 비동기 변형인 `aget_relevant_documents()`)를 구현합니다.

`Retriever`의 가장 일반적인 유형은 벡터 저장소의 유사성 검색 기능을 사용해 검색을 용이하게 하는 `VectorStoreRetriever`입니다. 모든 `VectorStore`는 `Retriever`로 쉽게 변환할 수 있습니다:

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
retrieved_docs = retriever.get_relevant_documents(
    "What are the approaches to Task Decomposition?"
)

In [ ]:
len(retrieved_docs)

In [ ]:
print(retrieved_docs[0].page_content)

## Step 5. Generate

질문을 받고, 관련 문서를 검색하고, 프롬프트를 구성하고, 이를 모델에 전달하고, 출력을 파싱하는 체인으로 이 모든 것을 합쳐 보겠습니다.

여기서는 `gpt-3.5-turbo` OpenAI 채팅 모델을 사용하겠지만, 어떤 LangChain 'LLM' 또는 'ChatModel'로 대체할 수 있습니다.

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

We'll use a prompt for RAG that is checked into the LangChain prompt hub ([here](https://smith.langchain.com/hub/rlm/rag-prompt)).

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
print(
    prompt.invoke(
        {"context": "filler context", "question": "filler question"}
    ).to_string()
)

우리는 체인을 정의하기 위해 [LCEL Runnable](https://python.langchain.com/docs/expression_language/) 프로토콜을 사용하여 다음과 같이 할 수 있습니다.

- 컴포넌트와 함수를 투명한 방식으로 파이프하고
- 스트리밍, 비동기, 일괄 호출을 바로 사용할 수 있습니다.

In [ ]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

#### Customizing the prompt

위와 같이 프롬프트 허브에서 프롬프트(예: [이 RAG 프롬프트](https://smith.langchain.com/hub/rlm/rag-prompt))를 로드할 수 있습니다. 프롬프트는 쉽게 사용자 지정할 수도 있습니다:

In [ ]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

### Adding sources

LCEL을 사용하면 검색된 문서 또는 문서에서 특정 소스 메타데이터를 쉽게 반환할 수 있습니다:

In [ ]:
from operator import itemgetter

from langchain.schema.runnable import RunnableParallel

rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

rag_chain_with_source.invoke("What is Task Decomposition")

### Adding memory

과거 사용자 입력을 기억하는 상태 저장 애플리케이션을 만들고 싶다고 가정해 보겠습니다. 이를 지원하기 위해 필요한 작업은 크게 두 가지입니다.
1. 과거 메시지를 전달할 수 있는 메시지 플레이스홀더를 체인에 추가합니다.
2. 최신 사용자 쿼리를 가져와 채팅 기록의 맥락에서 리트리버에 전달할 수 있는 독립형 질문으로 재형성하는 체인을 추가합니다.

2부터 시작하겠습니다. 다음과 같이 `condense question` 체인을 구축할 수 있습니다:

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [ ]:
from langchain.schema.messages import AIMessage, HumanMessage

condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

In [ ]:
condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "How do transformers work",
    }
)

이제 전체 QA 체인을 구축할 수 있습니다. 채팅 기록이 비어 있지 않을 때만 `condense question chain`을 실행하도록 라우팅 기능을 추가한 것을 주목하세요.

In [ ]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [ ]:
chat_history = []

question = "What is Task Decomposition?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "What are common ways of doing it?"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

## 연습문제
### 1. WebBaseLoader 기반 QA
* 주어진 사이트의 정보를 이용해 QA Agent를 구성해보자

[https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/](https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/)

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",),
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    },
)
docs = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
retrieved_docs = retriever.get_relevant_documents(
    "What are the approaches to Task Decomposition?"
)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
print(
    prompt.invoke(
        {"context": "filler context", "question": "filler question"}
    ).to_string()
)

In [ ]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)